In [1]:
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import RidgeClassifier
from sklearn.utils.extmath import softmax

#import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

#import matplotlib.pyplot as plt
#import seaborn as sns

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'claim'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 2017
    CVSEED = 2017
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 300
    VERBOSE = 1000
    SEED = 2017
    CVSEED = 2017
    #N_ITERS = 10

In [3]:
def set_seed(seed=2017):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

In [4]:
class RidgeClassifierwithProba(RidgeClassifier):
    def predict_proba(self, X):
        d = self.decision_function(X)
        d_2d = np.c_[-d, d]
        return softmax(d_2d)

# Datasets

In [5]:
INPUT = Path("../input/tabular-playground-series-sep-2021")

train = pd.DataFrame(pd.read_csv(INPUT / "train.csv")[target])
#test = pd.read_csv(INPUT / "test.csv")
#submission = pd.read_csv(INPUT / "sample_solution.csv")

In [6]:
train['lgb_pred'] = np.load("../input/tps-sep-lv0-base-trees/agg_lgb_oof.npy")
#test['lgb_pred'] = np.load("../input/tps-sep-lv0-base-trees/agg_lgb_pred.npy")

train['lgb2_pred'] = np.load("../input/tps-sep-lv0-base-trees/agg_lgb2_oof.npy")
#test['lgb2_pred'] = np.load("../input/tps-sep-lv0-base-trees/agg_lgb2_pred.npy")

train['xgb_pred'] = np.load("../input/tps-sep-lv0-base-trees/agg_xgb_oof.npy")
#test['xgb_pred'] = np.load("../input/tps-sep-lv0-base-trees/agg_xgb_pred.npy")

train['lgb_bizen_pred'] = np.load("../input/tps-sep-lv0-base-trees/lgb_bizen_oof.npy")
#test['lgb_bizen_pred'] = np.load("../input/tps-sep-lv0-base-trees/lgb_bizen_pred.npy")

train['lgb_dmitry_pred'] = np.load("../input/tps-sep-lv0-base-trees/lgb_dmitry_oof.npy")
#test['lgb_dmitry_pred'] = np.load("../input/tps-sep-lv0-base-trees/lgb_dmitry_pred.npy")

train['xgb_dmitry_pred'] = np.load("../input/tps-sep-lv0-base-trees/xgb_dmitry_oof.npy")
#test['xgb_dmitry_pred'] = np.load("../input/tps-sep-lv0-base-trees/xgb_dmitry_pred.npy")

train['lgb_manav_pred'] = np.load("../input/tps-sep-lv0-base-trees/lgb_manav_oof.npy")
#test['lgb_manav_pred'] = np.load("../input/tps-sep-lv0-base-trees/lgb_manav_pred.npy")

train['xgb_manav_pred'] = np.load("../input/tps-sep-lv0-base-trees/xgb_manav_oof.npy")
#test['xgb_manav_pred'] = np.load("../input/tps-sep-lv0-base-trees/xgb_manav_pred.npy")


# Preprocessing

In [7]:
features = [col for col in train.columns if 'pred' in col]

In [8]:
ss = StandardScaler()

In [9]:
train[features]

,lgb_pred,lgb2_pred,xgb_pred,lgb_bizen_pred,lgb_dmitry_pred,xgb_dmitry_pred,lgb_manav_pred,xgb_manav_pred
0,0.569058,0.572014,0.563386,0.567237,0.584811,0.563556,0.568376,0.580805
1,0.177095,0.162172,0.180320,0.167740,0.176388,0.169896,0.166929,0.164803
2,0.819233,0.817878,0.824730,0.819344,0.812119,0.818159,0.805830,0.823776
3,0.800054,0.792651,0.815384,0.800839,0.812413,0.782764,0.812170,0.793517
4,0.759385,0.762114,0.758896,0.778639,0.761375,0.766125,0.766569,0.792921
...,...,...,...,...,...,...,...,...
957914,0.136522,0.134028,0.139478,0.137695,0.137452,0.142344,0.133318,0.133796
957915,0.819584,0.803354,0.817831,0.792144,0.810890,0.817595,0.799856,0.809614
957916,0.110399,0.111084,0.107138,0.114974,0.111724,0.104979,0.113801,0.112733
957917,0.534861,0.554930,0.561236,0.544862,0.555083,0.565263,0.558625,0.549495


In [10]:
train[target]

0         1
1         0
2         1
3         1
4         1
         ..
957914    0
957915    1
957916    0
957917    1
957918    0
Name: claim, Length: 957919, dtype: int64

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
def objective(trial, X=train[features], y=train[target]):
 
  param_space = { 
              'max_iter':N_ESTIMATORS,
               'tol':1e-8,
        
               'alpha':trial.suggest_loguniform('alpha', 3.92545, 4.42545),
              'solver':trial.suggest_categorical('solver',['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])
                }
            

  seed_list=[SEED]

  kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  ridge_oof = np.zeros(train.shape[0])
  #ridge_pred = np.zeros(test.shape[0])
  
  #X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  
  for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
      print(f"===== fold {fold} =====")
   
      X_train = train[features].iloc[trn_idx]
      y_train = train[target].iloc[trn_idx]
      X_valid = train[features].iloc[val_idx]
      y_valid = train[target].iloc[val_idx]
      #X_test = test[features]
        
      
        
      X_train[features] = ss.fit_transform(X_train[features])
      X_valid[features] = ss.transform(X_valid[features])
      #X_test[features] = ss.transform(X_test[features])


      start = time.time()
      for inseed in seed_list:
            param_space['random_state'] = inseed

            model = RidgeClassifierwithProba(**param_space)
            model.fit(
                X_train, 
                y_train,
                
            )

            #ridge_oof[val_idx] = model.predict_proba(X_valid)
            
            ridge_oof[val_idx] += model.predict_proba(X_valid)[:,-1] / len(seed_list) 

      elapsed = time.time() - start
      auc = roc_auc_score(y_valid, ridge_oof[val_idx])

      print(f"fold {fold} - ridge rmse: {auc:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  auc_oof = roc_auc_score(train[target], ridge_oof)
  print(f"oof ridge_rmse = {roc_auc_score(train[target], ridge_oof)}")
  
  return auc_oof

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-09-20 00:14:10,095] A new study created in memory with name: no-name-23e20037-00d5-425f-bf11-5d7b4232a07e


===== fold 0 =====
fold 0 - ridge rmse: 0.817406, elapsed time: 0.80sec

===== fold 1 =====
fold 1 - ridge rmse: 0.816385, elapsed time: 0.49sec

===== fold 2 =====
fold 2 - ridge rmse: 0.817831, elapsed time: 0.46sec

===== fold 3 =====
fold 3 - ridge rmse: 0.816564, elapsed time: 0.47sec

===== fold 4 =====
fold 4 - ridge rmse: 0.818207, elapsed time: 0.46sec

===== fold 5 =====
fold 5 - ridge rmse: 0.815613, elapsed time: 0.48sec

===== fold 6 =====
fold 6 - ridge rmse: 0.819605, elapsed time: 0.47sec

===== fold 7 =====
fold 7 - ridge rmse: 0.815746, elapsed time: 0.45sec

===== fold 8 =====
fold 8 - ridge rmse: 0.817335, elapsed time: 0.47sec

===== fold 9 =====
fold 9 - ridge rmse: 0.816518, elapsed time: 0.50sec



[I 2021-09-20 00:14:19,811] Trial 0 finished with value: 0.8171108275942343 and parameters: {'alpha': 3.9280302308968844, 'solver': 'svd'}. Best is trial 0 with value: 0.8171108275942343.


oof ridge_rmse = 0.8171108275942343
===== fold 0 =====
fold 0 - ridge rmse: 0.817406, elapsed time: 0.44sec

===== fold 1 =====
fold 1 - ridge rmse: 0.816385, elapsed time: 0.43sec

===== fold 2 =====
fold 2 - ridge rmse: 0.817831, elapsed time: 0.39sec

===== fold 3 =====
fold 3 - ridge rmse: 0.816565, elapsed time: 0.43sec

===== fold 4 =====
fold 4 - ridge rmse: 0.818208, elapsed time: 0.42sec

===== fold 5 =====
fold 5 - ridge rmse: 0.815613, elapsed time: 0.47sec

===== fold 6 =====
fold 6 - ridge rmse: 0.819605, elapsed time: 0.41sec

===== fold 7 =====
fold 7 - ridge rmse: 0.815746, elapsed time: 0.39sec

===== fold 8 =====
fold 8 - ridge rmse: 0.817335, elapsed time: 0.41sec

===== fold 9 =====
fold 9 - ridge rmse: 0.816518, elapsed time: 0.49sec



[I 2021-09-20 00:14:28,719] Trial 1 finished with value: 0.8171108266831625 and parameters: {'alpha': 4.354133246689307, 'solver': 'sparse_cg'}. Best is trial 0 with value: 0.8171108275942343.


...

oof ridge_rmse = 0.8171108385576117
===== fold 0 =====
fold 0 - ridge rmse: 0.817406, elapsed time: 25.83sec

===== fold 1 =====
fold 1 - ridge rmse: 0.816385, elapsed time: 19.28sec

===== fold 2 =====
fold 2 - ridge rmse: 0.817831, elapsed time: 18.23sec

===== fold 3 =====
fold 3 - ridge rmse: 0.816565, elapsed time: 21.42sec

===== fold 4 =====
fold 4 - ridge rmse: 0.818208, elapsed time: 19.37sec

===== fold 5 =====
fold 5 - ridge rmse: 0.815613, elapsed time: 19.53sec

===== fold 6 =====
fold 6 - ridge rmse: 0.819605, elapsed time: 19.93sec

===== fold 7 =====
fold 7 - ridge rmse: 0.815746, elapsed time: 23.93sec

===== fold 8 =====
fold 8 - ridge rmse: 0.817335, elapsed time: 21.00sec

===== fold 9 =====
fold 9 - ridge rmse: 0.816518, elapsed time: 23.34sec



[I 2021-09-20 00:34:13,467] Trial 28 finished with value: 0.8171108337145451 and parameters: {'alpha': 4.309302156075652, 'solver': 'sag'}. Best is trial 22 with value: 0.8171108387189018.


oof ridge_rmse = 0.8171108337145451
===== fold 0 =====
fold 0 - ridge rmse: 0.817406, elapsed time: 0.35sec

===== fold 1 =====
fold 1 - ridge rmse: 0.816385, elapsed time: 0.33sec

===== fold 2 =====
fold 2 - ridge rmse: 0.817831, elapsed time: 0.32sec

===== fold 3 =====
fold 3 - ridge rmse: 0.816565, elapsed time: 0.32sec

===== fold 4 =====
fold 4 - ridge rmse: 0.818208, elapsed time: 0.32sec

===== fold 5 =====
fold 5 - ridge rmse: 0.815613, elapsed time: 0.32sec

===== fold 6 =====
fold 6 - ridge rmse: 0.819605, elapsed time: 0.34sec

===== fold 7 =====
fold 7 - ridge rmse: 0.815746, elapsed time: 0.32sec

===== fold 8 =====
fold 8 - ridge rmse: 0.817335, elapsed time: 0.32sec

===== fold 9 =====
fold 9 - ridge rmse: 0.816518, elapsed time: 0.33sec



[I 2021-09-20 00:34:21,111] Trial 29 finished with value: 0.8171108358810653 and parameters: {'alpha': 4.253592724699712, 'solver': 'auto'}. Best is trial 22 with value: 0.8171108387189018.


oof ridge_rmse = 0.8171108358810653
Number of finished trials: 30
Best trial: {'alpha': 4.213645840939098, 'solver': 'auto'}


In [14]:
study.best_params

{'alpha': 4.213645840939098, 'solver': 'auto'}

# Log

0.817110830994407 ver2

0.8171108348915279 ver3

0.8171108375419188 ver4

0.817110838788649 ver5

0.8171108396038185 ver6
